In [1]:
from pyhdf.SD import SD, SDC
import numpy as np
import matplotlib.pyplot as plt
import pprint
from os import listdir
import time
import h5py
import pandas as pd
from datetime import datetime, date, timedelta
import os
import sys

In [2]:
%matplotlib auto

Using matplotlib backend: TkAgg


In [3]:
def write_file(n_array, starttime, endtime):
    fpath = "D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/cloud/reduced"
    os.chdir(fpath)
    year = endtime.strftime('%Y')
    path = '-'.join((starttime.strftime('%m%d'), endtime.strftime('%m%d')))
    if not os.path.exists(year):#this is the first file being created for that time
        os.makedirs(year)
        #set index to 0

    path = '/'.join((year,path))
    path = '.'.join((path,"h5"))
    print path
    with h5py.File(str(path).replace("pkl","h5"), mode="w") as f:
        f.create_dataset("data", data=n_array.to_records(index=False))

In [6]:
file_name = 'D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/cloud/2018/summer/MYD06_L2.A2018152.0020.061.2018152170514.hdf'
file = SD(file_name, SDC.READ)

In [7]:
datasets_dic = file.datasets()

for idx,sds in enumerate(datasets_dic.keys()):
    print idx,sds

0 Cloud_Top_Height_Nadir_Night
1 Single_Scatter_Albedo_Ice
2 Radiance_Variance
3 Brightness_Temperature
4 Sensor_Azimuth
5 Statistics_1km_sds
6 Cloud_Water_Path_1621
7 Cloud_Phase_Infrared_Day
8 Cloud_Effective_Radius_16_PCL
9 Sensor_Zenith
10 Cloud_Water_Path_1621_PCL
11 cloud_top_method_1km
12 Tropopause_Height
13 Cloud_Water_Path
14 Cloud_Top_Pressure_Nadir_Day
15 Longitude
16 IRW_Low_Cloud_Temperature_From_COP
17 Cloud_Fraction_Nadir
18 Cloud_Top_Pressure_From_Ratios
19 Atm_Corr_Refl
20 Cirrus_Reflectance
21 Cloud_Top_Height_Nadir_Day
22 Cloud_Optical_Thickness_37_PCL
23 Cloud_Optical_Thickness_Uncertainty
24 Cloud_Effective_Radius
25 Cloud_Effective_Radius_PCL
26 Cloud_Fraction_Day
27 Cloud_Top_Pressure_Day
28 Cloud_Optical_Thickness_16_PCL
29 Cloud_Effective_Radius_1621_PCL
30 Solar_Zenith
31 cloud_emiss12_1km
32 Sensor_Azimuth_Day
33 Latitude
34 cloud_emissivity_1km
35 Cloud_Top_Temperature_Night
36 Sensor_Zenith_Night
37 Scan_Start_Time
38 Cloud_Optical_Thickness_1621
39 Cloud_

In [8]:
lat_obj = file.select('Latitude') # select sds
lat = lat_obj.get()

lon_obj = file.select('Longitude') # select sds
lon = lon_obj.get()

data_obj = file.select('Cloud_Fraction') # select sds
data = data_obj.get()/100.

starttime_obj = file.select('Scan_Start_Time') # select sds
starttime = starttime_obj.get()

brightness_temp_obj = file.select('Brightness_Temperature')
brightness_temp = brightness_temp_obj.get()

cloud_optical_thickness_obj = file.select('Cloud_Optical_Thickness')
cloud_optical_thickness = cloud_optical_thickness_obj.get()

quality_assurance_obj = file.select('Quality_Assurance_5km')
quality_assurance = quality_assurance_obj.get()

cloud_water_path_obj = file.select('Cloud_Water_Path')
cloud_water_path = cloud_water_path_obj.get()


In [9]:
pprint.pprint( data_obj.attributes() )

{'Cell_Across_Swath_Sampling': [3, 1348, 5],
 'Cell_Along_Swath_Sampling': [3, 2028, 5],
 'Geolocation_Pointer': 'Internal geolocation arrays',
 'Parameter_Type': 'Output',
 '_FillValue': 127,
 'add_offset': 0.0,
 'long_name': 'Cloud Fraction in Retrieval Region (5x5 1-km Pixels) from 1-km Cloud Mask',
 'scale_factor': 0.009999999776482582,
 'units': 'none',
 'valid_range': [0, 100]}


In [8]:
plt.figure()
plt.plot(data)

In [10]:
lonmin = lon.min()
lonmax = lon.max()
latmin = lat.min()
latmax = lat.max()
print lonmin, lonmax
print latmin, latmax
print len(lon), len(lat)

-179.70644 179.83208
65.42532 89.933624
406 406


In [11]:
delta = 0.1
rows, cols = np.where((abs(lat - 72.) <= delta) & (abs(lon - -38) <= delta))

In [13]:
latsub = lat[rows][:,cols]
lonsub = lon[rows][:,cols]
datasub = data[rows][:,cols]
#qualitysub = quality[rows][:,cols]

In [14]:
plt.figure()
plt.pcolormesh(lon, lat, data, cmap='jet')
plt.colorbar(label = 'cloud fraction')
plt.clim(0.,datasub.max())
#plt.axis([lonmin, lonmax, latmin, latmax])
plt.ylabel('latitude (deg)')
plt.xlabel('longitude (deg)')
#plt.title('%s %s data binned to azimuth and revolution #' % (name, var))
plt.grid()
plt.show()

ValueError: zero-size array to reduction operation maximum which has no identity

In [15]:
latstepdeg = 1.
longsetdeg = 1.
startday = 1
scalefactor = 100.

In [16]:
summit_lon = -38.4592
summit_lat = 72.5796

In [6]:
y = 2018
season = 'winter'
directory = 'D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/cloud/%d/%s/' % (y, season)
files = listdir(directory)

cloud_fraction = []
TIME = []
quality_assurance = []
latitude = []
longitude = []
brightness_temp = []
water_path = []
optical_thickness = []
numfiles = 0

print 'total number of files: %d' % len(files)
t1 = time.time()
for f in files:
    
    numfiles += 1
    
    try:
        file = SD(directory+f, SDC.READ)
    except:
        print 'passed: %s' % f
        continue
        
    lat_obj = file.select('Latitude') # select sds
    lat = lat_obj.get()

    lon_obj = file.select('Longitude') # select sds
    lon = lon_obj.get()
    
    delta = 0.05
    rows, cols = np.where((abs(lat - summit_lat) <= delta) & (abs(lon - summit_lon) <= delta))

    if len(rows) == 0 or len(cols) == 0:
        continue
        #print '# files: %d, ' % numfiles, f


    data_obj = file.select('Cloud_Fraction') # select sds
    data = data_obj.get()/100.

    starttime_obj = file.select('Scan_Start_Time') # select sds
    starttime = starttime_obj.get()

    #brightness_temp_obj = file.select('Brightness_Temperature')
    #cloud_brightness_temp = brightness_temp_obj.get()

    cloud_optical_thickness_obj = file.select('Cloud_Optical_Thickness')
    cloud_optical_thickness = cloud_optical_thickness_obj.get()

    quality_assurance_obj = file.select('Quality_Assurance_5km')
    quality = quality_assurance_obj.get()

    cloud_water_path_obj = file.select('Cloud_Water_Path')
    cloud_water_path = cloud_water_path_obj.get()
    
    datasub = data[rows][:,cols]
    timesub = starttime[rows][:,cols]
    qualitysub = quality[rows][:,cols]
    lonsub = lon[rows][:,cols]
    latsub = lat[rows][:,cols]
    #btempsub = cloud_brightness_temp[rows][:,cols]
    othicksub = cloud_optical_thickness[rows][:,cols]
    wpathsub = cloud_water_path[rows][:,cols]
    
    
    cloud_fraction.append(datasub[datasub>=0.].mean())
    #brightness_temp.append(btempsub[btempsub>=0.].mean())
    optical_thickness.append(othicksub[othicksub>=0.].mean())
    water_path.append(wpathsub[wpathsub>=0.].mean())
    TIME.append(timesub.mean())
    quality_assurance.append(qualitysub)
    latitude.append(latsub)
    longitude.append(lonsub)
    
        
    sys.stdout.write("\rPercent complete: %.2f, time elapsed: %.2f min" % ((float(numfiles) * 100. / float(len(files))), (time.time()-t1)/60.))
    sys.stdout.flush()
        
t2 = time.time()
print t2-t1
cloud_fraction = np.asarray(cloud_fraction)
TIME = np.asarray(TIME)
#brightness_temp = np.asarray(brightness_temp)
water_path = np.asarray(water_path)
optical_thickness = np.asarray(optical_thickness)
latitude = np.asarray(latitude)
longitude = np.asarray(longitude)
quality_assurance = np.asarray(quality_assurance)

total number of files: 1957
passed: checksums_501552069
Percent complete: 0.26, time elapsed: 0.03 min

C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: Mean of empty slice.
C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:73: RuntimeWarning: Mean of empty slice.


Percent complete: 99.54, time elapsed: 14.99 min904.067000151


In [149]:
lat_diff = []
for i in range(len(latitude)):
    if len(latitude[i]) > 1:
        for j in range(len(latitude[i])):
            if len(latitude[i][j]) > 1:
                for k in range(len(latitude[i][j])):
                    lat_diff.append(latitude[i][j][k] - summit_lat)
            else:
                lat_diff.append(latitude[i][j] - summit_lat)
    else:
        lat_diff.append(latitude[i] - summit_lat)
        
lat_diff = np.asarray(lat_diff)

In [150]:
print lat_diff[1]

0.03175864257812577


In [151]:
plt.figure()
plt.plot(lat_diff)
plt.ylabel('lat diff')
plt.title('max diff = %.2f, min diff = %.2f' % (lat_diff.max(), lat_diff.min()))

Text(0.5,1,'max diff = 0.05, min diff = -0.05')

In [28]:
plt.figure()
plt.plot(TIME, cloud_fraction)

In [7]:
doysecoffset= 23*60*60*24*365 + 7*60*60*24 - 17*60*60 - 60 - 5.761243
tt = TIME + doysecoffset


In [8]:
h5data = pd.DataFrame({"cloud fraction" : cloud_fraction})
h5data["time"] = tt
h5data["water path"] = water_path 
h5data["optical thickness"] = optical_thickness 
#h5data["latitude"] = latitude 
#h5data["longitude"] = longitude 
#h5data["quality assurance"] = quality_assurance 

In [9]:
starttime = datetime.fromtimestamp(tt[0])
endtime = datetime.fromtimestamp(tt[-1])
print starttime
print endtime

2017-12-01 03:18:43.544487
2018-02-28 12:57:41.398113


In [10]:
write_file(h5data, starttime, endtime)

2018/1201-0228.h5


In [4]:
TIME = {}
pwv = {}
cloudy_fraction = {}
year = [2018]

for y in year:
    fpath = "D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/cloud/reduced/%s/" % y
    files = listdir(fpath)
    TIME[y] = {}
    pwv[y] = {}
    cloudy_fraction[y] = {}
    for f in files:
        dset = h5py.File(fpath+f, 'r')['data']
        
        if f[:2] == '12':
            season = 'winter'
        else:
            season = 'summer'
            
        cloudy_fraction[y][season] = dset['cloud fraction']
        TIME[y][season] = dset['time']
        #pwv[y][season] = dset['pwv']

In [5]:
y = 2018
s = 'winter'
plt.figure()
plt.plot(cloudy_fraction[y][s])

In [6]:
cloudmean = {}
tcloudmean = {}
season = ['winter']
i0=0
for y in year:
    tcloudmean[y] = {}
    cloudmean[y] = {}
    for s in season:
        cf = cloudy_fraction[y][s][np.logical_not(np.isnan(cloudy_fraction[y][s]))]
        ti = TIME[y][s][np.logical_not(np.isnan(TIME[y][s]))]
        cmean = []
        tmean = []
        t = TIME[y][s]
        for h in range(len(t)):
            if t[h] - t[i0] >= 3600*24:
                cmean.append(cf[i0:h+1].mean())
                tmean.append(ti[i0:h+1].mean())
                i0=h+1
            if i0<len(t):
                cmean.append(cf[i0:len(t)+1].mean())
                tmean.append(ti[i0:len(t)+1].mean())
        i0 = 0
        cloudmean[y][s] = np.asarray(cmean)
        tcloudmean[y][s] = np.asarray(tmean)

In [9]:
print datetime.fromtimestamp(tcloudmean[2018]['winter'][-1])

2018-02-28 12:57:41.398113


In [10]:
for y in year:
    for s in season:
        print y, s
        #print len(pwvmean[y][s])
        print len(tcloudmean[y][s])
        print len(cloudmean[y][s])
        print '---------------'

2018 winter
475
475
---------------


In [ ]:
plt.figure()
plt.plot(cloudmean[2018]['winter'])

In [23]:
print cloudy_fraction[y][s].shape, cloudmean[y][s].shape

(409L,) (475L,)


In [29]:
scale = 1.
bins=50
plt.figure()
i = 0
plt.suptitle('Summit cloudy fraction, Prob(cloudy fraction $>$ x)')

for y in year:
    
    #plt.subplot(3, 2, i+1)
    i += 1
    for s in season:
        print y, s
        values, base = np.histogram(cloudy_fraction[y][s], bins=bins, density=True)
        cumulative = np.cumsum(values)
        cumu = cumulative/cumulative.max()
        cumu_I = 1-cumu
        if s == 'winter':
            plt.plot(base[:-1]*scale, cumu_I*100, '--', linewidth = 2, label = ' %s %d' % (s, y))
        else:
            plt.plot(base[:-1]*scale, cumu_I*100, linewidth = 2, label = ' %s %d' % (s, y))
    
    if y >= 2017:
        plt.xlabel('cloudy sky fraction')
    if (y % 2) == 1:
        plt.ylabel('Cumulative Distribution (%)')
    #plt.ylim([0, 80])
    plt.xlim([0, 1.1])
    plt.legend()
    plt.grid()
plt.show()

2018 winter


In [171]:
print season

winter


In [173]:
scale = 1.
bins=50
plt.figure()
i = 0
plt.suptitle('Summit PWV daily averages, Prob(cloudy fraction $>$ x)')

for y in year:
    
    plt.subplot(3, 2, i+1)
    i += 1
    for s in season:
        print y, s
        values, base = np.histogram(pwvmean[y][s], bins=bins, density=True)
        cumulative = np.cumsum(values)
        cumu = cumulative/cumulative.max()
        cumu_I = 1. - cumu
        if s == 'winter':
            plt.plot(base[:-1]*scale, cumu_I*100, '--', linewidth = 2, label = ' %s %d' % (s, y))
        else:
            plt.plot(base[:-1]*scale, cumu_I*100, linewidth = 2, label = ' %s %d' % (s, y))
    
    if y >= 2017:
        plt.xlabel('PWV (mm)')
    if (y % 2) == 1:
        plt.ylabel('Cumulative Distribution (%)')
    #plt.ylim([0, 80])
    plt.xlim([0, 1.1])
    plt.legend()
    plt.grid()
plt.show()

2013 summer
2013 winter
2014 summer
2014 winter
2015 summer
2015 winter
2016 summer
2016 winter
2017 summer
2017 winter
2018 summer
2018 winter


In [177]:
print pwvmean[2018]['summer']
print pwvmean[2018]['winter'].shape

[1.500625   2.49055556 2.29916667]
(10L,)


In [163]:
#y = 2018
bins = 50
minimum = 0.5
maximum = 1
plt.figure()
for i in range(len(year)):
    plt.subplot(3, 2, i+1)
    s = 'summer'
    plt.hist(cloudmean[year[i]][s], bins = bins, range = (minimum, maximum), label='%s %d' % (s, year[i])) 
    s = 'winter'
    plt.hist(cloudmean[year[i]][s], bins = bins, range = (minimum, maximum), label='%s %d' % (s, year[i])) 
    #if i >= 2:
    plt.xlabel('cloudy sky fraction')
    #if i == 0 or i == 2:
    plt.ylabel('counts')
    plt.legend()
    plt.grid()